**Imports**

In [1]:
import torch
import torch.nn as nn
import numpy as np
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F

**Device Configuration**

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

**HyperParameters**

In [3]:
num_epochs = 10
batch_size = 4
learning_rate = 0.001

**Dataset**

In [4]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_data = torchvision.datasets.CIFAR10(root="/content/sample_data", train=True, transform=transform, download=True)
test_data = torchvision.datasets.CIFAR10(root="/content/sample_data", train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_data, shuffle=True, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(dataset=test_data, shuffle=False, batch_size=batch_size)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


**CNN Model**

In [14]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
    self.fc1 = nn.Linear(in_features=16*5*5, out_features=120)
    self.fc2 = nn.Linear(in_features=120, out_features=84)
    self.fc3 = nn.Linear(in_features=84, out_features=10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16*5*5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x) # no activation as it is in loss
    return x

model = ConvNet().to(device)  

**Loss and Optimizer**

In [15]:
criterion = nn.CrossEntropyLoss()   # softmax in already included here
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

**Training Loops**

In [16]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    # forward pss
    outputs = model(images)
    loss = criterion(outputs, labels)

    # backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i%1000)==0:
      print(f'epoch {epoch+1} / {num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')

epoch 1 / 10, step 1/12500, loss = 2.3071
epoch 1 / 10, step 1001/12500, loss = 2.3309
epoch 1 / 10, step 2001/12500, loss = 2.2861
epoch 1 / 10, step 3001/12500, loss = 2.3354
epoch 1 / 10, step 4001/12500, loss = 2.2841
epoch 1 / 10, step 5001/12500, loss = 2.3021
epoch 1 / 10, step 6001/12500, loss = 2.2553
epoch 1 / 10, step 7001/12500, loss = 2.2271
epoch 1 / 10, step 8001/12500, loss = 2.2451
epoch 1 / 10, step 9001/12500, loss = 2.3165
epoch 1 / 10, step 10001/12500, loss = 2.0950
epoch 1 / 10, step 11001/12500, loss = 2.3436
epoch 1 / 10, step 12001/12500, loss = 2.1755
epoch 2 / 10, step 1/12500, loss = 1.8764
epoch 2 / 10, step 1001/12500, loss = 2.2274
epoch 2 / 10, step 2001/12500, loss = 1.9493
epoch 2 / 10, step 3001/12500, loss = 2.0694
epoch 2 / 10, step 4001/12500, loss = 1.9651
epoch 2 / 10, step 5001/12500, loss = 2.5590
epoch 2 / 10, step 6001/12500, loss = 2.1700
epoch 2 / 10, step 7001/12500, loss = 2.6031
epoch 2 / 10, step 8001/12500, loss = 1.6993
epoch 2 / 10,

**Model Evaluation**

In [19]:
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  n_class_correct = [0 for i in range(10)]
  n_class_samples = [0 for i in range(10)]
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs, 1)
    n_samples += labels.shape[0]
    n_correct += (predicted==labels).sum().item()

    for i in range(batch_size):
      label = labels[i]
      pred = predicted[i]
      if label==pred:
        n_class_correct[label] += 1
      n_class_samples[label] += 1


acc = 100*(n_correct/n_samples)
print("Accuracy on test data is :", acc)

Accuracy on test data is : 56.25
